# 04 - dataset blancement

In [ ]:
# Importing libraries

import os
import random
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from shutil import copy2, rmtree
from scipy.ndimage import zoom
from typing import Dict, List, Tuple, Union